### Comparison of two ways to compute the sensitivity matrix used in the equivalent layer composed by a set of  dipoles with unit magnetic moment 
### By Valeria Barbosa

In [1]:
import numpy as np

from datetime import datetime

from fatiando.vis import mpl
from fatiando import mesher, gridder, utils
from fatiando.mesher import Prism, PointGrid
from fatiando.utils import dircos
from fatiando.gravmag import prism, sphere


C:\Anaconda2\lib\site-packages\fatiando\vis\mpl.py:70: UserWarning: This module will be removed in v0.6. We recommend the use of matplotlib.pyplot module directly. Some of the fatiando specific functions will remain.
  "specific functions will remain.")


In [2]:
# 
# Function to compute the sensitivity matrix of the Total Field anomaly caused by magnetized DIPOLE 
# x axis is north, z axis is down
#
# Input parameters:
# Observation point located at (xp,yp,zp). 
#
# Dipole centered at (xc,yc,zc)  
# Magnetization of dipole is defined by the direction cossines mx_s, my_s, mz_s
#
# The Geomagnetic Field is defined by the direction cossines: MX_Geomag, MY_Geomag, MZ_Geomag
#
# ATENTION:  THE dipole HAS: UNIT VOLUME AND UNIT  MAGNETIC MOMENT 
#
# The Units:
# of distance irrelevant but must be consistent. 
#
# Output parameters:
# tf = Total field anomaly in units of nT.
#
# Adapted from function Dipole of Blakely (1995)


def dipole_unit_moment(xp, yp, zp, xc, yc, zc, mx_s, my_s, mz_s, MX_Geomag, MY_Geomag, MZ_Geomag):
    #: Proportionality constant used in the magnetic method in henry/m (SI)
    CM = 10. ** (-7)
    #: Conversion factor from tesla to nanotesla
    T2NT = 10. ** (9)
    
    rx = xp - xc
    ry = yp - yc
    rz = zp - zc
    r2 = rx*rx + ry*ry + rz*rz
    r  = np.sqrt(r2)
    if np.any(r==0): print "Bad argument detected"
    r5 = r**5
    dot = rx*mx_s + ry*my_s + rz*mz_s
    volume = 1.0
    moment = 1.0
    # moment = 4.*np.pi*(a**3)*m/3.
    
    bx = T2NT*CM*moment*(3.*dot*rx-r2*mx_s)/r5
    by = T2NT*CM*moment*(3.*dot*ry-r2*my_s)/r5
    bz = T2NT*CM*moment*(3.*dot*rz-r2*mz_s)/r5
    
    tf = bx*MX_Geomag + by*MY_Geomag + bz*MZ_Geomag
    return tf


# DIRCOS computes direction cosines from inclination and declination,
#
# Input parameters:
# incl: inclination in degrees positive below horizontal,
# decl: declination in degrees positive east of true north,
# azim: azimuth of x axis in degrees positive east of north,
#
# Output parameters:
# mx,my,mz: the three direction cosines


def dircos_blakely(incl,decl,azim):
    d2rad = 0.017453293
    xincl=incl*d2rad
    xdecl=decl*d2rad
    xazim=azim*d2rad
    mx=np.cos(xincl)*np.cos(xdecl-xazim)
    my=np.cos(xincl)*np.sin(xdecl-xazim)
    mz=np.sin(xincl)
    return mx, my, mz


In [3]:
# 
# Function to compute the sensitivity matrix of the Bx By Bz caused by magnetized DIPOLE 
# x axis is north, z axis is down
#
# Input parameters:
# Observation point located at (xp,yp,zp). 
#
# Dipole centered at (xc,yc,zc)  
# Magnetization of dipole is defined by the direction cossines mx_s, my_s, mz_s
#
# The Geomagnetic Field is defined by the direction cossines: MX_Geomag, MY_Geomag, MZ_Geomag
#
# ATENTION:  THE dipole HAS: UNIT VOLUME AND UNIT  MAGNETIC MOMENT 
#
# The Units:
# of distance irrelevant but must be consistent. 
#
# Output parameters:
# bx by and bz   in units of nT.
#
# Adapted from function Dipole of Blakely (1995)
def dipole_BX_BY_BZ_unit_moment(xp, yp, zp, xc, yc, zc, mx_s, my_s, mz_s):
    #: Proportionality constant used in the magnetic method in henry/m (SI)
    CM = 10. ** (-7)
    #: Conversion factor from tesla to nanotesla
    T2NT = 10. ** (9)
    
    rx = xp - xc
    ry = yp - yc
    rz = zp - zc
    r2 = rx*rx + ry*ry + rz*rz
    r  = np.sqrt(r2)
    if np.any(r==0): print "Bad argument detected"
    r5 = r**5
    dot = rx*mx_s + ry*my_s + rz*mz_s
    volume = 1.0
    moment = 1.0
    # moment = 4.*np.pi*(a**3)*m/3.
    
    bx = T2NT*CM*moment*(3.*dot*rx-r2*mx_s)/r5
    by = T2NT*CM*moment*(3.*dot*ry-r2*my_s)/r5
    bz = T2NT*CM*moment*(3.*dot*rz-r2*mz_s)/r5
    
    return bx, by, bz


# Grid of Observations

In [4]:
start_time_total = datetime.now()

N_East    = 100
N_North   = 100
area      = (-5000., 5000., -5000., 5000.)
shape_obs = (N_North,N_East)

z_dado = 0.0

xi, yi, zi = gridder.regular(area, shape_obs, z=z_dado)


In [5]:
### Input 

# Inclination and declination of the of the Geomagnetic Field

inc =  20.
dec =  10.

# Inclination and declination of the equivalent sources are DIFFERENT to Inclination and declintion of the true source dec_eq = decs

inc_eq = 60.
dec_eq = 20.


# Equivalent sources (using the same shape of the data)

In [6]:
plano_fonte = 200.
zk = np.zeros_like(zi) + plano_fonte
print zk

xk, yk, zk = gridder.regular(area, shape_obs, z=zk)

[ 200.  200.  200. ...,  200.  200.  200.]


## By Valeria:  Sensitivity Matrix using dipoles 

In [7]:
Npts     = len(xi)
M_source = len(zk)
print Npts, M_source 


TF = np.empty((Npts,M_source),dtype =float)

#  Direction cosines of the equivalent layer
# The three direction cosines of the dipoles that set up the equivalent layer 
mx_eq, my_eq, mz_eq = dircos_blakely(inc_eq, dec_eq,0.)


#  Direction cosines of the Geomagnetic Field
# The three direction cosines of the of the Geomagnetic Field
MX, MY, MZ = dircos_blakely(inc,dec,0.)


# Computing the sensitivity matrix by column 
start_time_matrizes_valeria = datetime.now()

for j  in range(M_source):
    TF[:,j] = dipole_unit_moment(xi,yi,zi,xk[j],yk[j],zk[j], mx_eq, my_eq, mz_eq, MX, MY, MZ)


end_time_matrizes_valeria = datetime.now()
print('Duration: {}'.format(end_time_matrizes_valeria - start_time_matrizes_valeria))



10000 10000
Duration: 0:00:11.302000


In [8]:
print 'Inclination and declination of the Geomagnetic Field: ', inc, dec
print 'direction cosines of the Geomagnetic Field: ',  MX, MY, MZ

print 'Inclination and declination of the Eq Layer (in degrees):', inc_eq, dec_eq
print 'direction cosines of the dipoles that set up the equivalent layer:', mx_eq, my_eq, mz_eq 

Inclination and declination of the Geomagnetic Field:  20.0 10.0
direction cosines of the Geomagnetic Field:  0.925416574381 0.163175915039 0.342020152348
Inclination and declination of the Eq Layer (in degrees): 60.0 20.0
direction cosines of the dipoles that set up the equivalent layer: 0.469846285311 0.171010067642 0.866025418186


## By FATIANDO: Sensitivity Matrix using dipoles

In [9]:
# Sensitivity Matrix from Fatiando
mag_eq = dircos(inc_eq, dec_eq)

print 'Inclination and declination of the Eq Layer (in degrees):', inc_eq, dec_eq
print 'direction cosines of the dipoles that set up the equivalent layer: ' ,  mag_eq




Inclination and declination of the Eq Layer (in degrees): 60.0 20.0
direction cosines of the dipoles that set up the equivalent layer:  [0.46984631039295432, 0.17101007166283438, 0.8660254037844386]


In [10]:
# creating the layer of the equivalent layer
layer = PointGrid(area, zk, shape_obs)

start_time_matrizes_fatiando = datetime.now()

TF_Fatiando = np.empty((Npts, M_source),dtype =float)
for i, c in enumerate(layer):
    TF_Fatiando[:,i] = sphere.tf(xi, yi, zi, [c], inc, dec, pmag = mag_eq)
    
end_time_matrizes_fatiando = datetime.now()
print('Duration: {}'.format(end_time_matrizes_fatiando - start_time_matrizes_fatiando))
    
   

Duration: 0:00:05.892000


In [11]:
np.allclose(TF_Fatiando, TF)

True

#  Sensitivity Matricies of Bx By and Bz using dipoles 

### By Valeria: Sensitivity Matrices of BX BY and BZ using dipoles

In [12]:
BX = np.empty((Npts, M_source),dtype =float)
BY = np.empty((Npts, M_source),dtype =float)
BZ = np.empty((Npts, M_source),dtype =float)


# Computing the sensitivity matrix by column 
start_time_matrizes_componentes_valeria = datetime.now()

for j  in range(M_source):
    BX[:,j], BY[:,j],BZ[:,j] = dipole_BX_BY_BZ_unit_moment(xi,yi,zi,xk[j],yk[j],zk[j], mx_eq, my_eq, mz_eq)


end_time_matrizes_valeria = datetime.now()
print('Duration: {}'.format(end_time_matrizes_valeria - start_time_matrizes_valeria))


Duration: 0:00:35.131000


### By FATIANDO: Sensitivity Matrices of BX BY and BZ using dipoles

In [13]:
# creating the layer of the equivalent layer
layer = PointGrid(area, zk, shape_obs)

start_time_matrizes_componentes_fatiando = datetime.now()

BX_Fatiando = np.empty((Npts, M_source),dtype =float)
BY_Fatiando = np.empty((Npts, M_source),dtype =float)
BZ_Fatiando = np.empty((Npts, M_source),dtype =float)

for i, c in enumerate(layer):
    BX_Fatiando[:,i] = sphere.bx(xi, yi, zi, [c], pmag = mag_eq)
    BY_Fatiando[:,i] = sphere.by(xi, yi, zi, [c], pmag = mag_eq)
    BZ_Fatiando[:,i] = sphere.bz(xi, yi, zi, [c], pmag = mag_eq)
    
end_time_matrizes_componentes_fatiando = datetime.now()
print('Duration: {}'.format(end_time_matrizes_componentes_fatiando - start_time_matrizes_componentes_fatiando))
    

Duration: 0:00:13.467000


In [15]:
np.allclose(BX_Fatiando, BX)


True

In [16]:
np.allclose(BY_Fatiando, BY)


True

In [17]:
np.allclose(BZ_Fatiando, BZ)

True